In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import cifar10

In [2]:
# set gpu
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype("float32")/255.0                           # normalize it
x_test = x_test.astype("float32")/255.0

In [4]:
model = keras.Sequential([
    keras.Input(shape=(32, 32, 3)),
    layers.Conv2D(filters=32, kernel_size=3, padding='valid', activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10),
])

In [5]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.0003),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],)

C:\Users\crazyguy\miniconda3\envs\tfod\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [6]:
model.fit(x_train, y_train, batch_size=64,epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/10
782/782 - 12s - loss: 1.6808 - accuracy: 0.3858 - 12s/epoch - 16ms/step
Epoch 2/10
782/782 - 7s - loss: 1.3318 - accuracy: 0.5249 - 7s/epoch - 10ms/step
Epoch 3/10
782/782 - 8s - loss: 1.1931 - accuracy: 0.5774 - 8s/epoch - 10ms/step
Epoch 4/10
782/782 - 8s - loss: 1.1013 - accuracy: 0.6148 - 8s/epoch - 10ms/step
Epoch 5/10
782/782 - 8s - loss: 1.0171 - accuracy: 0.6445 - 8s/epoch - 10ms/step
Epoch 6/10
782/782 - 8s - loss: 0.9592 - accuracy: 0.6653 - 8s/epoch - 10ms/step
Epoch 7/10
782/782 - 10s - loss: 0.9116 - accuracy: 0.6826 - 10s/epoch - 13ms/step
Epoch 8/10
782/782 - 10s - loss: 0.8641 - accuracy: 0.6981 - 10s/epoch - 12ms/step
Epoch 9/10
782/782 - 12s - loss: 0.8270 - accuracy: 0.7133 - 12s/epoch - 15ms/step
Epoch 10/10
782/782 - 9s - loss: 0.7884 - accuracy: 0.7271 - 9s/epoch - 12ms/step
157/157 - 1s - loss: 0.9068 - accuracy: 0.6829 - 1s/epoch - 9ms/step


[0.9068309664726257, 0.6829000115394592]

In [7]:
def my_model():
    inputs = keras.Input(shape=(32,32,3))
    x = layers.Conv2D(32, 3)(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    
    x = layers.Conv2D(64, 5, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    #x = layers.MaxPooling2D()(x)
    
    x = layers.Conv2D(128, 3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(10)(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


model = my_model()

model.compile(
    optimizer=keras.optimizers.Adam(lr=0.0003),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],)

model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [20]:
# there is a gap between training and test accuracy, we define this model is overfitting / underfitting to the training data
# we reduce the overfitting/underfitting by method regularisation 

In [4]:
# CNN -> BatchNorm -> ReLU

In [8]:
def my_model():
    inputs = keras.Input(shape=(32,32,3))
    x = layers.Conv2D(32, 3, padding='same', kernel_regularizer=regularizers.L2(0.01))(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    
    x = layers.Conv2D(64, 5, padding='same', kernel_regularizer=regularizers.L2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    #x = layers.MaxPooling2D()(x)
    
    x = layers.Conv2D(128, 3, padding='same', kernel_regularizer=regularizers.L2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.L2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10)(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


model = my_model()

model.compile(
    optimizer=keras.optimizers.Adam(lr=0.0003),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],)

model.fit(x_train, y_train, batch_size=64,epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/10
782/782 - 29s - loss: 3.0809 - accuracy: 0.1697 - 29s/epoch - 37ms/step
Epoch 2/10
782/782 - 32s - loss: 2.2125 - accuracy: 0.1879 - 32s/epoch - 42ms/step
Epoch 3/10
782/782 - 33s - loss: 2.0983 - accuracy: 0.2067 - 33s/epoch - 42ms/step
Epoch 4/10
782/782 - 33s - loss: 2.0429 - accuracy: 0.2272 - 33s/epoch - 42ms/step
Epoch 5/10
782/782 - 27s - loss: 1.9915 - accuracy: 0.2355 - 27s/epoch - 34ms/step
Epoch 6/10
782/782 - 28s - loss: 1.9773 - accuracy: 0.2362 - 28s/epoch - 36ms/step
Epoch 7/10
782/782 - 29s - loss: 1.9569 - accuracy: 0.2433 - 29s/epoch - 38ms/step
Epoch 8/10
782/782 - 30s - loss: 1.9472 - accuracy: 0.2436 - 30s/epoch - 38ms/step
Epoch 9/10
782/782 - 30s - loss: 1.9384 - accuracy: 0.2444 - 30s/epoch - 38ms/step
Epoch 10/10
782/782 - 30s - loss: 1.9335 - accuracy: 0.2469 - 30s/epoch - 38ms/step
157/157 - 2s - loss: 1.9595 - accuracy: 0.2591 - 2s/epoch - 10ms/step


[1.9594900608062744, 0.2590999901294708]